In [95]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import MSELoss
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [96]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [97]:
raw_2019_10 = pd.read_csv('/content/drive/MyDrive/MEng AI/AI 201/Mini-Project/data/2019-Oct.csv')
raw_2019_11 = pd.read_csv('/content/drive/MyDrive/MEng AI/AI 201/Mini-Project/data/2019-Nov.csv')
raw_2019_12 = pd.read_csv('/content/drive/MyDrive/MEng AI/AI 201/Mini-Project/data/2019-Dec.csv')
raw_2020_01 = pd.read_csv('/content/drive/MyDrive/MEng AI/AI 201/Mini-Project/data/2020-Jan.csv')
raw_2020_02 = pd.read_csv('/content/drive/MyDrive/MEng AI/AI 201/Mini-Project/data/2020-Feb.csv')

In [98]:
raw_data = raw_2019_10.append(raw_2019_10)
raw_data = raw_data.append(raw_2019_11)
raw_data = raw_data.append(raw_2019_12)
raw_data = raw_data.append(raw_2020_01)
raw_data = raw_data.append(raw_2020_02)

<ipython-input-98-96fd9db230bf>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_2019_10.append(raw_2019_10)
<ipython-input-98-96fd9db230bf>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_data.append(raw_2019_11)
<ipython-input-98-96fd9db230bf>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_data.append(raw_2019_12)
<ipython-input-98-96fd9db230bf>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  raw_data = raw_data.append(raw_2020_01)
<ipython-input-98-96fd9db230bf>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

In [99]:
print("raw_2019_10 shape: {}" .format(raw_2019_10.shape))
print("raw_2019_11 shape: {}" .format(raw_2019_11.shape))
print("raw_2019_12 shape: {}" .format(raw_2019_12.shape))
print("raw_2020_01 shape: {}" .format(raw_2020_01.shape))
print("raw_2020_02 shape: {}" .format(raw_2020_02.shape))

raw_2019_10 shape: (4102283, 9)
raw_2019_11 shape: (4635837, 9)
raw_2019_12 shape: (3533286, 9)
raw_2020_01 shape: (4264752, 9)
raw_2020_02 shape: (4156682, 9)


In [100]:
raw_data.shape

(24795123, 9)

In [101]:
raw_data.isnull().sum()

event_time              0
event_type              0
product_id              0
category_id             0
category_code    24374052
brand            10416378
price                   0
user_id                 0
user_session         5235
dtype: int64

In [102]:
raw_data = raw_data.dropna(subset = ['category_code'])

In [103]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421071 entries, 21 to 4156566
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   event_time     421071 non-null  object 
 1   event_type     421071 non-null  object 
 2   product_id     421071 non-null  int64  
 3   category_id    421071 non-null  int64  
 4   category_code  421071 non-null  object 
 5   brand          315434 non-null  object 
 6   price          421071 non-null  float64
 7   user_id        421071 non-null  int64  
 8   user_session   421012 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 32.1+ MB


In [104]:
raw_data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
21,2019-10-01 00:00:53 UTC,view,5856191,1487580006350586771,appliances.environment.vacuum,runail,24.44,507355498,944c7e9b-40bd-4112-a05b-81e73f37e0c0
264,2019-10-01 00:07:58 UTC,remove_from_cart,5767493,1487580013053083824,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
269,2019-10-01 00:08:03 UTC,remove_from_cart,5759489,1487580013053083824,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
274,2019-10-01 00:08:08 UTC,remove_from_cart,5657894,1487580013053083824,stationery.cartrige,NaN,2.06,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
276,2019-10-01 00:08:10 UTC,remove_from_cart,24336,1487580013053083824,stationery.cartrige,depilflax,3.02,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1


In [105]:
raw_data['event_type'].unique()

array(['view', 'remove_from_cart', 'cart', 'purchase'], dtype=object)

In [106]:
#change categorical variables into binary values
raw_data['event_type'].replace({'view':0, 'remove_from_cart':0, 'purchase':1,'cart':1}, inplace = True)

In [107]:
raw_data['brand'] = raw_data['brand'].fillna('unknown')

In [108]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421071 entries, 21 to 4156566
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   event_time     421071 non-null  object 
 1   event_type     421071 non-null  int64  
 2   product_id     421071 non-null  int64  
 3   category_id    421071 non-null  int64  
 4   category_code  421071 non-null  object 
 5   brand          421071 non-null  object 
 6   price          421071 non-null  float64
 7   user_id        421071 non-null  int64  
 8   user_session   421012 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 32.1+ MB


In [109]:
raw_data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
21,2019-10-01 00:00:53 UTC,0,5856191,1487580006350586771,appliances.environment.vacuum,runail,24.44,507355498,944c7e9b-40bd-4112-a05b-81e73f37e0c0
264,2019-10-01 00:07:58 UTC,0,5767493,1487580013053083824,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
269,2019-10-01 00:08:03 UTC,0,5759489,1487580013053083824,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
274,2019-10-01 00:08:08 UTC,0,5657894,1487580013053083824,stationery.cartrige,unknown,2.06,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
276,2019-10-01 00:08:10 UTC,0,24336,1487580013053083824,stationery.cartrige,depilflax,3.02,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1


In [110]:
raw_data.groupby('event_type')['user_id'].count()

event_type
0    318491
1    102580
Name: user_id, dtype: int64

In [111]:
raw_data.groupby('user_id')['product_id'].count().sort_values(ascending = False)

user_id
400419007    620
558545016    414
557616099    388
461327949    370
531900924    301
            ... 
581127752      1
581127820      1
581127848      1
581127858      1
622090014      1
Name: product_id, Length: 92172, dtype: int64

In [112]:
conditions = [
    (raw_data['price'] <= 4.71), #1st quartile
    (raw_data['price'] <= 24.6), #median
    (raw_data['price'] <= 63.33), #3rd quartile
    (raw_data['price'] > 63.33) #4th quartile
]

values = ['1st quartile','2nd quartile','3rd quartile', '4th quartile']

raw_data['price_2'] = np.select(conditions,values)

print(raw_data)

                      event_time  event_type  product_id          category_id  \
21       2019-10-01 00:00:53 UTC           0     5856191  1487580006350586771   
264      2019-10-01 00:07:58 UTC           0     5767493  1487580013053083824   
269      2019-10-01 00:08:03 UTC           0     5759489  1487580013053083824   
274      2019-10-01 00:08:08 UTC           0     5657894  1487580013053083824   
276      2019-10-01 00:08:10 UTC           0       24336  1487580013053083824   
...                          ...         ...         ...                  ...   
4156389  2020-02-29 23:47:54 UTC           0     5861766  1487580006350586771   
4156487  2020-02-29 23:51:55 UTC           0     5774248  2193074740619379535   
4156552  2020-02-29 23:54:28 UTC           0     5856193  2193074740619379535   
4156562  2020-02-29 23:54:43 UTC           0     5925386  2235524499216793855   
4156566  2020-02-29 23:54:44 UTC           1     5929649  1487580006350586771   

                         ca

In [113]:
features = ['user_id','event_type','product_id', 'category_code', 'price_2']
#features = ['event_type','product_id','category_id','category_code','brand', 'price_2']

In [114]:
raw_data_v2 = raw_data[features]

In [115]:
raw_data_v2

,user_id,event_type,product_id,category_code,price_2
21,507355498,0,5856191,appliances.environment.vacuum,2nd quartile
264,514753614,0,5767493,stationery.cartrige,1st quartile
269,514753614,0,5759489,stationery.cartrige,1st quartile
274,514753614,0,5657894,stationery.cartrige,1st quartile
276,514753614,0,24336,stationery.cartrige,1st quartile
...,...,...,...,...,...
4156389,586536651,0,5861766,appliances.environment.vacuum,1st quartile
4156487,458687218,0,5774248,furniture.living_room.cabinet,4th quartile
4156552,458687218,0,5856193,furniture.living_room.cabinet,4th quartile
4156562,622090014,0,5925386,appliances.personal.massager,4th quartile


In [116]:
features_v2 = (raw_data_v2.drop(columns=['event_type'])).columns
features_v2

Index(['user_id', 'product_id', 'category_code', 'price_2'], dtype='object')

In [117]:
features_v2 = (raw_data_v2.drop(columns=['event_type'])).columns
idx = {feature:None for feature in features_v2}

In [118]:
for feature in features_v2 :
    feature2idx = {v:k for k,v in enumerate(raw_data[feature].unique())}
    idx[feature] = len(feature2idx)
    raw_data_v2[feature] = raw_data_v2[feature].map(feature2idx)
raw_data_v2.head(5)

<ipython-input-118-718602345d83>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data_v2[feature] = raw_data_v2[feature].map(feature2idx)
<ipython-input-118-718602345d83>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data_v2[feature] = raw_data_v2[feature].map(feature2idx)
<ipython-input-118-718602345d83>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,user_id,event_type,product_id,category_code,price_2
21,0,0,0,0,0
264,1,0,1,1,1
269,1,0,2,1,1
274,1,0,3,1,1
276,1,0,4,1,1


In [119]:
idx

{'user_id': 92172, 'product_id': 624, 'category_code': 12, 'price_2': 4}

**Defining an FFM Model**

In [120]:
class FeaturesLinear(torch.nn.Module) :
    def __init__(self, field_dims) :
        '''
        Parameter
            field_dims : List of field dimensions
        '''
        super().__init__()
        self.input_dim = sum(field_dims)
        self.linear = nn.Linear(self.input_dim, 1, bias=True)
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int32)

    def forward(self, x):
        '''
        Parameter
            x : Long tensor of size (batch_size, num_fields)

        Return
            linear_term : Float tensor of size (batch_size)
        '''
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        sparse_x = torch.zeros(x.size(0), self.input_dim, device=x.device).scatter_(1, x, 1.)
        linear_term = self.linear(sparse_x)
        return linear_term

In [121]:
class FieldAwareEmbedding(torch.nn.Module) :

    def __init__(self, field_dims, embed_dim) :
        '''
        Parameter
            field_dims : List of field dimensions
            embed_dim : Factorization dimension for dense embedding
        '''
        super().__init__()
        self.num_fields = len(field_dims)
        self.embeddings = nn.ModuleList([
            nn.Embedding(feature_size, embed_dim) for feature_size in field_dims
        ])
        for embedding in self.embeddings:
            torch.nn.init.xavier_uniform_(embedding.weight.data)

    def forward(self, x) :
        '''
        Parameter
            x : Long tensor of size (batch_size, num_fields)

        Return
            dense_x : Long tensor of size (batch_size, num_fields, embed_dim)
        '''
        dense_x = [self.embeddings[i](x[..., i]) for i in range(self.num_fields)]
        dense_x = torch.stack(dense_x, dim=1)
        return dense_x

In [122]:
class FieldAwareFactorizationMachine(nn.Module) :
    def __init__(self, field_dims, embed_dim) :
        '''
        Parameter
            field_dims : List of field dimensions
            embed_dim : Factorization dimension for dense embedding
        '''
        super().__init__()
        self.num_fields = len(field_dims)
        self.embeddings = FieldAwareEmbedding(field_dims, embed_dim)
        self.linear = FeaturesLinear(field_dims)

    def square(self, x):
        return torch.pow(x,2)

    def forward(self, x) :
        '''
        Parameter
            x : Long tensor of size (batch_size, num_fields)

        Return
            y_ffm : Float tensor of size (batch_size)
        '''
        linear_term = self.linear(x)

        dense_x = self.embeddings(x)

        square_of_sum = self.square(torch.sum(dense_x, dim=1))
        sum_of_square = torch.sum(self.square(dense_x), dim=1)
        pairwise_term = 0.5 * torch.sum(square_of_sum - sum_of_square, dim=1)

        y_ffm = linear_term.squeeze(1) + pairwise_term

        return y_ffm

**Create a Dataset**

In [123]:
X = raw_data_v2.drop(['event_type'], axis = 1).values
y = raw_data_v2['event_type'].values

In [124]:
#Apply SMOTE

from imblearn.over_sampling import RandomOverSampler, SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X,y)

In [125]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size = 0.2, random_state = 5, shuffle = True, stratify =y)

print("X_train_valid shape: {}" .format(X_train_valid.shape))
print("X_test.shape: {}" .format(X_test.shape))
print("y_train_valid.shape: {}" .format(y_train_valid.shape))
print("y_test.shape: {}" .format(y_test.shape))

X_train_valid shape: (509585, 4)
X_test.shape: (127397, 4)
y_train_valid.shape: (509585,)
y_test.shape: (127397,)


In [126]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size = 0.2, random_state = 5, shuffle = True, stratify =y_train_valid)

print("X_train shape: {}" .format(X_train.shape))
print("X_valid.shape: {}" .format(X_valid.shape))
print("y_train.shape: {}" .format(y_train.shape))
print("y_valid.shape: {}" .format(y_valid.shape))

X_train shape: (407668, 4)
X_valid.shape: (101917, 4)
y_train.shape: (407668,)
y_valid.shape: (101917,)


In [127]:
train_dataset = TensorDataset(torch.LongTensor(X_train),
                              torch.LongTensor(y_train))
valid_dataset = TensorDataset(torch.LongTensor(X_valid),
                              torch.LongTensor(y_valid))
test_dataset = TensorDataset(torch.LongTensor(X_test),
                             torch.LongTensor(y_test))

In [128]:
train_dataset

**Define Evaluation Functions**

In [129]:
class MultiClassCrossEntropyLoss(nn.Module):
    def __init__(self):
        super(MultiClassCrossEntropyLoss, self).__init__()
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x, y):
        # Assuming x is the output from your model (logits) and y is the ground truth label
        loss = self.criterion(x, y)
        return loss


def model_train(model, dataloader, loss_fn, optimizer, device) :
    model.train()
    total_loss = 0
    correct_predictions = 0
    roc_auc = 0
    i = 0

    total_samples = 0
    for features, targets in tqdm(dataloader) :
        features = features.to(device)
        targets = targets.to(device).to(torch.float32)
        
        outputs = model(features)
        loss = loss_fn(targets, outputs)

        #FOR TESTING
        predicted_probabilities = torch.sigmoid(outputs)
        predicted_classes = torch.round(predicted_probabilities)

        threshold = 0.5  # or any other desired probability threshold
        predicted_classes = (predicted_probabilities > threshold).float()
        correct_predictions += (predicted_classes == targets).sum().item()
        # print("targets: {}" .format(targets))
        # print("outputs: {}" .format(outputs))
        # print("predicted_classes: {}" .format(predicted_classes))
        # print("correct_predictions: {}" .format(correct_predictions))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * targets.size(0)
        total_samples += len(targets) #added

        i = i + 1
    #accuracy = (correct_predictions / len(dataloader.dataset))*100
    accuracy = (correct_predictions / total_samples)*100
    avg_loss = total_loss / len(dataloader.dataset)
    return avg_loss, accuracy

In [130]:
from sklearn.metrics import roc_auc_score, f1_score
import numpy as np

def model_train(model, dataloader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    all_true_labels = []
    all_predicted_probs = []

    for features, targets in tqdm(dataloader):
        features = features.to(device)
        targets = targets.to(device).to(torch.float32)
        outputs = model(features)
        loss = loss_fn(targets, outputs)
        predicted_probabilities = torch.sigmoid(outputs)


        # Collect probabilities and true labels
        all_true_labels.extend(targets.cpu().numpy().astype(int))  # Ensure labels are integers
        all_predicted_probs.extend(predicted_probabilities.detach().cpu().numpy())
        total_loss += loss* targets.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Convert probabilities to binary predictions using a threshold (0.5 for binary classification)
    predicted_labels = np.round(all_predicted_probs)

    accuracy = (np.sum(predicted_labels == all_true_labels) / len(all_true_labels)) * 100
    avg_loss = total_loss / len(dataloader.dataset)
    # Compute ROC AUC and F1 Score after collecting all predictions
    roc_auc = roc_auc_score(all_true_labels, all_predicted_probs)
    f1 = f1_score(all_true_labels, predicted_labels, average='binary')
    recall = recall_score(all_true_labels, predicted_labels, average='binary')
    precision = precision_score(all_true_labels, predicted_labels, average='binary')
    return avg_loss, accuracy, roc_auc, f1, recall,precision

# Note: The original code doesn't compute 'correct_predictions' correctly here,
# so I've replaced its usage with a direct comparison of predicted_labels and all_true_labels.


def model_eval(model, dataloader, loss_fn, device) :
    with torch.no_grad() :
        model.eval()
        total_loss = 0
        correct_predictions = 0
        total_samples = 0
        all_true_labels = []
        all_predicted_probs = []

        for features, targets in tqdm(dataloader) :
            features = features.to(device)
            targets = targets.to(device).to(torch.float32)
            
            outputs = model(features)
            loss = loss_fn(targets, outputs)
            total_loss += loss

            #FOR TESTING
            predicted_probabilities = torch.sigmoid(outputs)
            predicted_classes = torch.round(predicted_probabilities)

            threshold = 0.5  # or any other desired probability threshold
            predicted_classes = (predicted_probabilities > threshold).float()

            # print("outputs: {}" .format(outputs))
            # print("predicted_classes: {}" .format(predicted_classes))
            # print("targets: {}" .format(targets))
            correct_predictions += (predicted_classes == targets).sum().item()
            total_samples += len(targets)
        accuracy = correct_predictions / total_samples * 100
        avg_loss = total_loss / len(dataloader)
    return avg_loss, accuracy

In [131]:
def model_eval(model, dataloader, loss_fn, device) :
    with torch.no_grad() :
        model.eval()
        total_loss = 0
        correct_predictions = 0
        total_samples = 0
        all_true_labels = []
        all_predicted_probs = []

        for features, targets in tqdm(dataloader) :
            features = features.to(device)
            targets = targets.to(device).to(torch.float32)

            outputs = model(features)
            loss = loss_fn(targets, outputs)
            predicted_probabilities = torch.sigmoid(outputs)
            total_loss += loss* targets.size(0)

            all_true_labels.extend(targets.cpu().numpy().astype(int))  # Ensure labels are integers
            all_predicted_probs.extend(predicted_probabilities.detach().cpu().numpy())


            #FOR TESTING
        predicted_labels = np.round(all_predicted_probs)
        accuracy = (np.sum(predicted_labels == all_true_labels) / len(all_true_labels)) * 100
        avg_loss = total_loss / len(dataloader.dataset)
        roc_auc = roc_auc_score(all_true_labels, all_predicted_probs)
        f1 = f1_score(all_true_labels, predicted_labels, average='binary')
        recall = recall_score(all_true_labels, predicted_labels, average='binary')
        precision = precision_score(all_true_labels, predicted_labels, average='binary')
            # print("outputs: {}" .format(outputs))
            # print("predicted_classes: {}" .format(predicted_classes))
            # print("targets: {}" .format(targets))

    return avg_loss, accuracy, roc_auc, f1, recall, precision

**Model Training**

In [132]:
field_dims = np.array(list(idx.values()), dtype=np.uint32)
print(field_dims)

[92172   624    12     4]


In [133]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [94]:
from torch.nn import BCEWithLogitsLoss

embed_dim = 4
batch_size = 128
#lr = 0.005
lr = 0.0000005
epochs = 30

ffm_model = FieldAwareFactorizationMachine(field_dims, embed_dim=embed_dim).to(device)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#loss_fn = RMSELoss()
#loss_fn = MultiClassCrossEntropyLoss()
loss_fn = BCEWithLogitsLoss()
optimizer = torch.optim.Adam(ffm_model.parameters(), lr=lr)

best_loss = int(1e8)
check_cnt = 0

for epoch in range(epochs) :
    train_loss, train_accuracy, train_roc_auc, train_f1, train_recall, train_precision= model_train(ffm_model, train_loader, loss_fn, optimizer, device)
    valid_loss, valid_accuracy, valid_roc_auc, valid_f1, valid_recall, valid_precision = model_eval(ffm_model, valid_loader, loss_fn, device)

    print(f"Epoch {epoch+1}")
    print(f"Train Loss : {train_loss:.2f},Train Accuracy : {train_accuracy:.2f}, Train ROC AUC : {train_roc_auc:.2f}, Train f1 : {train_f1:.2f},Train recall : {train_recall:.2f},Train precision : {train_precision:.2f},Validation Loss : {valid_loss:.2f}, Validation Accuracy: {valid_accuracy:.2f}%, Valid ROC AUC : {valid_roc_auc:.2f}, Valid f1 : {valid_f1:.2f}, Valid recall : {valid_recall:.2f},Valid precision : {valid_precision:.2f}")

    if best_loss > valid_loss :
        best_loss = valid_loss
        torch.save(ffm_model.state_dict(), 'ffm_model.pt')
        check_cnt = 0
    else :
        check_cnt += 1
        if check_cnt == 5 :
            print("Early Stopped")
            break

100%|██████████| 797/797 [00:35<00:00, 22.52it/s]


Epoch 1
Train Loss : 0.96,Train Accuracy : 63.52, Train ROC AUC : 0.66, Train f1 : 0.68,Train recall : 0.79,Train precision : 0.60,Validation Loss : 0.96, Validation Accuracy: 63.62%, Valid ROC AUC : 0.66, Valid f1 : 0.69, Valid recall : 0.79,Valid precision : 0.60


100%|██████████| 797/797 [00:35<00:00, 22.38it/s]


Epoch 2
Train Loss : 0.95,Train Accuracy : 63.63, Train ROC AUC : 0.67, Train f1 : 0.69,Train recall : 0.80,Train precision : 0.60,Validation Loss : 0.95, Validation Accuracy: 63.81%, Valid ROC AUC : 0.67, Valid f1 : 0.69, Valid recall : 0.80,Valid precision : 0.60


100%|██████████| 797/797 [00:34<00:00, 22.77it/s]


...

Epoch 28
Train Loss : 0.81,Train Accuracy : 53.60, Train ROC AUC : 0.69, Train f1 : 0.66,Train recall : 0.89,Train precision : 0.52,Validation Loss : 0.81, Validation Accuracy: 53.66%, Valid ROC AUC : 0.69, Valid f1 : 0.66, Valid recall : 0.89,Valid precision : 0.52


100%|██████████| 797/797 [00:36<00:00, 22.05it/s]


Epoch 29
Train Loss : 0.81,Train Accuracy : 53.57, Train ROC AUC : 0.69, Train f1 : 0.66,Train recall : 0.90,Train precision : 0.52,Validation Loss : 0.80, Validation Accuracy: 53.65%, Valid ROC AUC : 0.69, Valid f1 : 0.66, Valid recall : 0.90,Valid precision : 0.52


100%|██████████| 797/797 [00:36<00:00, 21.93it/s]


Epoch 30
Train Loss : 0.80,Train Accuracy : 53.53, Train ROC AUC : 0.69, Train f1 : 0.66,Train recall : 0.90,Train precision : 0.52,Validation Loss : 0.80, Validation Accuracy: 53.64%, Valid ROC AUC : 0.69, Valid f1 : 0.66, Valid recall : 0.90,Valid precision : 0.52


In [134]:
best_model = FieldAwareFactorizationMachine(field_dims, embed_dim).to(device)
best_model.load_state_dict(torch.load('ffm_model.pt'))

test_loss = model_eval(best_model, test_loader, loss_fn, device)
#print(f"Test Loss is {test_loss:.2f}")
print("Test Loss is: {}" .format(test_loss))

100%|██████████| 996/996 [00:14<00:00, 69.60it/s]


Test Loss is: (tensor(0.7984), 53.43767906622605, 0.6908802977416062, 0.6579795545356527, 0.8957597450509427, 0.5199566239588839)


In [135]:
!pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.2/265.2 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.44.0.3-py2.py3-none-any.whl size=265293968 sha256=7a683448f92fc18208ff5882bb566b74343f44901fe6d054430b0d0cd15aee01
  Stored in directory: /root/.cache/pip/wheels/77/9a/1c/2da26f943fd46b57f3c20b54847b936b9152b831dc7447cf71
Successfully built h2o


In [136]:
# Importing necessary packages
import h2o
from h2o.automl import H2OAutoML

# Initialize H2O
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.21" 2023-10-17; OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp_2a9r3kz
  JVM stdout: /tmp/tmp_2a9r3kz/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp_2a9r3kz/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,27 days
H2O_cluster_name:,H2O_from_python_unknownUser_igk9o6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [137]:
# Convert x_train and y_train to pandas DataFrames
x_train_df = pd.DataFrame(X_train, columns = raw_data_v2.drop(['event_type'], axis = 1).columns)
y_train_df = pd.DataFrame({'target': y_train})

In [138]:
# Merge x_train and y_train into a single DataFrame
train_df = pd.concat([x_train_df, y_train_df], axis = 1)

# Convert x_test and y_test to pandas DataFrames
x_test_df = pd.DataFrame(X_test, columns = raw_data_v2.drop(['event_type'], axis = 1).columns)
y_test_df = pd.DataFrame({'target': y_test})

# Merge x_test and y_test into a single DataFrame
test_df = pd.concat([x_test_df, y_test_df], axis = 1)

# Convert the pandas DataFrame to an H2OFrame
auto_train = h2o.H2OFrame(train_df)
auto_test = h2o.H2OFrame(test_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [139]:
auto_train['target'] = auto_train['target'].asfactor()
auto_test['target'] = auto_test['target'].asfactor()

In [140]:
x = auto_train.columns[:-1]
y = 'target'

In [141]:
model = H2OAutoML(max_runtime_secs=60, seed = 13)
model.train(x=x, y=y, training_frame = auto_train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,blending
Number of base models (used / total),2/3
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,AUTO
Metalearner nfolds,0
Metalearner fold_column,None
Custom metalearner hyperparameters,None


In [142]:
model.explain()

TypeError: explain() missing 1 required positional argument: 'frame'